In [1]:
import pandas as pd
import numpy as np
import os
import sys

# Exportar/importar modelos
import pickle

# Modelos
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor


In [3]:
# Importo el modelo desde el archivo .pickle
model = pickle.load(open('../output/models/quinto_procesado_quinto_modelo_a_xgboost.pickle', 'rb'))


In [4]:
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('estimator',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.1, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=5, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=0, num_parallel_tree=1,
                              predictor='auto', random_state=0, reg_alpha=0,
                              re

In [5]:
df_original = pd.read_csv('../data/raw_data/data.csv')
df_features_rellenas = pd.read_csv('../data/processed/quinto_procesado_a_entrenamiento.csv', sep='\t')
df_target = pd.read_csv('../data/processed/quinto_procesado_a_target.csv', sep='\t')
df_features_rellenas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21229 entries, 0 to 21228
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pressure [MPa]       21229 non-null  float64
 1   mass_flux [kg/m2-s]  21229 non-null  float64
 2   x_e_out [-]          21229 non-null  float64
 3   D_e [mm]             21229 non-null  float64
 4   D_h [mm]             21229 non-null  float64
 5   length [mm]          21229 non-null  float64
 6   chf_exp [MW/m2]      21229 non-null  float64
dtypes: float64(7)
memory usage: 1.1 MB


In [6]:
# Divido df_target en X e y. y es x_e_out [-]
X = df_target.drop('x_e_out [-]', axis = 1)
y = df_target['x_e_out [-]']
# Predigo con el modelo
y_pred = model.predict(X)


c:\Users\prado\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- D_e [mm]
- D_h [mm]
- chf_exp [MW/m2]
- length [mm]
- mass_flux [kg/m2-s]
- ...
Feature names seen at fit time, yet now missing:
- D_e
- D_h
- chf_exp
- length
- mass_flux
- ...

  warnings.warn(message, FutureWarning)


In [7]:
y_pred = pd.DataFrame(y_pred, columns = ['x_e_out [-]'])

In [8]:
# Añadimos el campo id, que coincide con el id de df_original donde x_e_out [-] es NaN
y_pred['id'] = df_original[df_original['x_e_out [-]'].isna()]['id'].values

In [9]:
# Cambiamos el orde de las columnas para que x_e_out [-] sea la última
y_pred = y_pred[['id', 'x_e_out [-]']]
y_pred.columns

Index(['id', 'x_e_out [-]'], dtype='object')

In [10]:
# Exportamos el dataframe con las predicciones para hacer submit
y_pred.to_csv('../output/submissions/5_submission.csv', index=False)